In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.plotting.strat_dashboard import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )

# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../data/prices.hd5')
rsi_ind = from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    candle_ohlc="close",
)

In [3]:
strat_results_df, settings_results_df = backtest_df_only(
    price_data=price_data,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=4,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=8,
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total candles per symbol: 9,408
Total candles to test: 752,640

Total combinations to test: 80


In [2]:
strat_results_df, settings_results_df = backtest_df_only(
    price_data=price_data,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=np.arange(4, 5.1, 1),
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(8, 9.1, 1),
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=np.arange(8, 9.1, 1),
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 32
Total order settings to test: 160
Total candles per symbol: 9,408
Total candles to test: 6,021,120

Total combinations to test: 640


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function mul>) found for signature:
 
 >>> mul(PriceTuple(float64 x 4), float64)
 
There are 14 candidate implementations:
[1m      - Of which 12 did not match due to:
      Overload of function 'mul': File: <numerous>: Line N/A.
        With argument(s): '(PriceTuple(float64 x 4), float64)':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Operator Overload in function 'mul': File: unknown: Line unknown.
        With argument(s): '(PriceTuple(float64 x 4), float64)':[0m
[1m       No match for registered cases:
        * (int64, int64) -> int64
        * (int64, uint64) -> int64
        * (uint64, int64) -> int64
        * 

In [ ]:
strat_results_df

In [ ]:
settings_results_df.iloc[:,10608]

In [ ]:
order_records = simulate_up_to_6_nb(
    prices=price_data['ETHUSD'].values,
    entries=entries.iloc[:,[11]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=8,
    risk_rewards=3,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=CandleBody.low,
    tsl_pcts_init=8,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [ ]:
entries.iloc[:,[11]]

In [ ]:
rsi_ind[[('ETHUSD', 30)]]

In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('ETHUSD', 30)]],
        "entries1": entries.iloc[:,[11]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.ETHUSD,
    order_records=order_records,
)